In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pyNastran
from pyNastran.op2.op2 import OP2

In [2]:
def ReadOP2Files(path):
    path = path + '*.op2'
    print('Given File path is : \n' , path , '\n')
    op2ListName = glob.glob(path)
    print('Found the following list of OP2 Files: \n')
    print('========================================', '\n')
    op2ListName
    dictOP2FileList = {}
    for i in range(len(op2ListName)):
        #print(colors[i])
        dictOP2FileList[os.path.splitext(os.path.basename(op2ListName[i]))[0]] = op2ListName[i]
        #print(os.path.basename(op2ListName[i]), op2ListName[i])
        #print(os.path.splitext(os.path.basename(op2ListName[i])))
    return dictOP2FileList

In [3]:
def ReadOP2File(op2FileName):
    preOP2Model = OP2()
    preOP2File = op2FileName
    # Load Pre OP2 File
    preOP2Model.read_op2(preOP2File, build_dataframe=True)
    return preOP2Model

In [4]:
def ReadLoadcases(preOP2Model):
#     print(preOP2Model.get_op2_stats(short=True))
    preLCs = list(preOP2Model.cshear_stress.keys())
    return preLCs

In [5]:
# Function Which creates new index for Shear Stress individual loadcase
def newShearIndex(lc):
    colReInx = pd.MultiIndex.from_tuples([(lc, 'max_shear'), (lc, 'avg_shear'), (lc, 'margin')])
    return colReInx

In [6]:
def ReadShearStressResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)
    # Start with an empty dataframe [note: Logic to be changed later]
    cShearResultDF_Pre = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cshear_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newShearIndex(lc), level=1)
        if cShearResultDF_Pre.empty == False:
            cShearResultDF_Pre = pd.concat([cShearResultDF_Pre, tempDF1], axis=1,sort=False)
        else :
            cShearResultDF_Pre = tempDF1.copy()
    return cShearResultDF_Pre

In [7]:
# Function Which creates new index for Bar Stress individual loadcase
def newBarIndex(lc):
    colReInx = pd.MultiIndex.from_tuples([(lc, 's1a'), (lc, 's2a'), (lc, 's3a'), (lc, 's4a'), (lc, 'axial'), (lc, 'smaxa'), (lc, 'smina'), (lc, 'MS_tension'), (lc, 's1b'), (lc, 's2b'), (lc, 's3b'), (lc, 's4b'), (lc, 'smaxb'), (lc, 'sminb'), (lc, 'MS_compression')])
    return colReInx

In [8]:
def ReadBarStressResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)
    # Start with an empty dataframe [note: Logic to be changed later]
    cBarResultDF_Pre = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cbar_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newBarIndex(lc), level=1)
        if cBarResultDF_Pre.empty == False:
            cBarResultDF_Pre = pd.concat([cBarResultDF_Pre, tempDF1], axis=1,sort=False)
        else :
            cBarResultDF_Pre = tempDF1.copy()
    return cBarResultDF_Pre
    

In [9]:
# Function Which creates new index for Bar Stress individual loadcase
def newBarIndex_Axial(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'axial')])
    return colReInx

In [10]:
def ReadBarAxialStressResuls(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)
    # Start with an empty dataframe [note: Logic to be changed later]
    cBarResultDF_Pre_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cbar_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newBarIndex_Axial(lc), level=1)
        if cBarResultDF_Pre_Axial.empty == False:
            cBarResultDF_Pre_Axial = pd.concat([cBarResultDF_Pre_Axial, tempDF1], axis=1,sort=False)
        else :
            cBarResultDF_Pre_Axial = tempDF1.copy()    
    return cBarResultDF_Pre_Axial

In [11]:
# Function Which creates new index for Quad Stress individual loadcase
def newQUADIndex_Fx(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'oxx')])
    return colReInx
# Function Which creates new index for Quad Stress individual loadcase
def newQUADIndex_Fy(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'oyy')])
    return colReInx
# Function Which creates new index for Quad Stress individual loadcase
def newQUADIndex_Fxy(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'txy')])
    return colReInx

In [12]:
def ReadShellFxResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)    
    # Start with an empty dataframe [note: Logic to be changed later]
    cShellResultDF_Pre_Fx = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cquad4_stress[lc].data_frame
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fx(lc), level=1)
        if cShellResultDF_Pre_Fx.empty == False:
            cShellResultDF_Pre_Fx = pd.concat([cShellResultDF_Pre_Fx, tempDF1], axis=1,sort=False)
        else :
            cShellResultDF_Pre_Fx = tempDF1.copy()    
    return cShellResultDF_Pre_Fx

In [13]:
def ReadShellFyResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)     
    # Start with an empty dataframe [note: Logic to be changed later]
    cShellResultDF_Pre_Fy = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cquad4_stress[lc].data_frame
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fy(lc), level=1)
        if cShellResultDF_Pre_Fy.empty == False:
            cShellResultDF_Pre_Fy = pd.concat([cShellResultDF_Pre_Fy, tempDF1], axis=1,sort=False)
        else :
            cShellResultDF_Pre_Fy = tempDF1.copy()
    return cShellResultDF_Pre_Fy    

In [14]:
def ReadShellFxyResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)    
    # Start with an empty dataframe [note: Logic to be changed later]
    cShellResultDF_Pre_Fxy = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cquad4_stress[lc].data_frame
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fxy(lc), level=1)
        if cShellResultDF_Pre_Fxy.empty == False:
            cShellResultDF_Pre_Fxy = pd.concat([cShellResultDF_Pre_Fxy, tempDF1], axis=1,sort=False)
        else :
            cShellResultDF_Pre_Fxy = tempDF1.copy()
    return cShellResultDF_Pre_Fxy

# QUAD and TRIA Combined Functions

In [15]:
def Read_Quad_and_Tria_Fx_Results(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)    
    # Start with an empty dataframe [note: Logic to be changed later]
    Combined_2D = pd.DataFrame({'A' : []})
    for lc in preLCs:
        quadDF = preOP2Model.cquad4_stress[lc].data_frame
        quadDF = Add_Type(quadDF, 'CQUAD4')
        triaDF = preOP2Model.ctria3_stress[lc].data_frame
        triaDF = Add_Type(triaDF, 'CTRIA3')
        tempDF = pd.concat([quadDF, triaDF], axis=0,sort=False)
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fx(lc), level=1)
        if Combined_2D.empty == False:
            Combined_2D = pd.concat([Combined_2D, tempDF1], axis=1,sort=False)
        else :
            Combined_2D = tempDF1.copy()
    return Combined_2D

def Read_Quad_and_Tria_Fy_Results(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)    
    # Start with an empty dataframe [note: Logic to be changed later]
    Combined_2D = pd.DataFrame({'A' : []})
    for lc in preLCs:
        quadDF = preOP2Model.cquad4_stress[lc].data_frame
        quadDF = Add_Type(quadDF, 'CQUAD4')
        triaDF = preOP2Model.ctria3_stress[lc].data_frame
        triaDF = Add_Type(triaDF, 'CTRIA3')
        tempDF = pd.concat([quadDF, triaDF], axis=0,sort=False)
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fy(lc), level=1)
        if Combined_2D.empty == False:
            Combined_2D = pd.concat([Combined_2D, tempDF1], axis=1,sort=False)
        else :
            Combined_2D = tempDF1.copy()    
    return Combined_2D

def Read_Quad_and_Tria_Fxy_Results(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)    
    # Start with an empty dataframe [note: Logic to be changed later]
    Combined_2D = pd.DataFrame({'A' : []})
    for lc in preLCs:
        quadDF = preOP2Model.cquad4_stress[lc].data_frame
        quadDF = Add_Type(quadDF, 'CQUAD4')
        triaDF = preOP2Model.ctria3_stress[lc].data_frame
        triaDF = Add_Type(triaDF, 'CTRIA3')
        tempDF = pd.concat([quadDF, triaDF], axis=0,sort=False)
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fxy(lc), level=1)
        if Combined_2D.empty == False:
            Combined_2D = pd.concat([Combined_2D, tempDF1], axis=1,sort=False)
        else :
            Combined_2D = tempDF1.copy()    
    return Combined_2D

In [16]:
# Function Which creates new index for Rod Axial Stress individual loadcase
def newRodIndex_Axial(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'axial')])
    return colReInx

In [17]:
def ReadRodAxialStressResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)     
    # Start with an empty dataframe [note: Logic to be changed later]
    cRodResultDF_Pre_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.crod_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newRodIndex_Axial(lc), level=1)
        if cRodResultDF_Pre_Axial.empty == False:
            cRodResultDF_Pre_Axial = pd.concat([cRodResultDF_Pre_Axial, tempDF1], axis=1,sort=False)
        else :
            cRodResultDF_Pre_Axial = tempDF1.copy()    
    return cRodResultDF_Pre_Axial

In [18]:
def AppendRodStressToBarStressDataFrame(cBarResultDF_Pre_Axial,cRodResultDF_Pre_Axial):
    Appended_Pre_Results = cBarResultDF_Pre_Axial.append(cRodResultDF_Pre_Axial, ignore_index=False)
    return Appended_Pre_Results

In [19]:
# ===========================================
# Main Start of OP2 Files Extraction == v0.0
# Remarks := Its been depricated.
# Author := Upendra Reddy G
# ===========================================

# dictOp2List = ReadOP2Files('./')
# for key, op2File in dictOp2List.items():
#     #print(key, op2File)
#     op2Model = ReadOP2File(op2File)

#     barStressFileName = key + '_cBar_Stress.csv'
#     barAxialStressFileName = key + '_cBar_Stress_Axial.csv'
#     quadStressFxFileName = key + '_cQUAD_Stress_Fx.csv'
#     quadStressFyFileName = key + '_cQUAD_Stress_Fy.csv'
#     quadStressFxyFileName = key + '_cQUAD_Stress_Fxy.csv'
#     rodAxialStressFileName = key + '_cRod_Stress_Axial.csv'
#     appendedBarRodstressFileName = key + '_Appended_cBar_and_Rod_Stresses_Axial.csv'
    
    
#     barStresResults = ReadBarStressResults(op2Model)
#     barStresResults.to_csv(barStressFileName)

#     barAxialStresResults = ReadBarAxialStressResuls(op2Model)
#     barAxialStresResults.to_csv(barAxialStressFileName)
    
#     quadFxStresResults = ReadShellFxResults(op2Model)
#     quadFxStresResults.to_csv(quadStressFxFileName)
    
#     quadFyStresResults = ReadShellFyResults(op2Model)
#     quadFyStresResults.to_csv(quadStressFyFileName)    
        
#     quadFxyStresResults = ReadShellFxyResults(op2Model)
#     quadFxyStresResults.to_csv(quadStressFxyFileName)
    
#     rodAxialStressResults = ReadRodAxialStressResults(op2Model)
#     rodAxialStressResults.to_csv(rodAxialStressFileName)
    
#     appendRodToBarResults = AppendRodStressToBarStressDataFrame(barAxialStresResults,rodAxialStressResults)
#     appendRodToBarResults.to_csv(appendedBarRodstressFileName)

# print(' ======== ALL DONE SUCCESSFULLY ========== ')

In [20]:
def Concatenate_Data(df1,df2):
    if df1.empty == False:
        df1 = pd.concat([df1, df2], axis=1,sort=False)
    else :
        df1 = df2.copy()
    return df1

In [21]:
def Join_DF(df1,df2):
    if df1.empty == True:
        df1 = df2.copy()
    else :
        df1 = df1.join(df2, how='outer')
    return df1

In [22]:
def newBush_Index(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'fx'), (lc, 'fy'), (lc, 'fz'), (lc, 'mx'), (lc, 'my'), (lc, 'mz') ])
    return colReInx
def ReadcBushForces(preOP2Model):
    preLCs = list(preOP2Model.cbush_force.keys())
    # Start with an empty dataframe [note: Logic to be changed later]
    cRodResultDF_Pre_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cbush_force[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newBush_Index(lc), level=1)
        if cRodResultDF_Pre_Axial.empty == False:
            cRodResultDF_Pre_Axial = pd.concat([cRodResultDF_Pre_Axial, tempDF1], axis=1,sort=False)
        else :
            cRodResultDF_Pre_Axial = tempDF1.copy()
    return cRodResultDF_Pre_Axial

In [23]:
def Add_Type(df, ElmType):
    df = df.reset_index()
    df['Type'] = ElmType
    df = df.set_index(['ElementID', 'NodeID', 'Location', 'Type'])
    return df

In [24]:
# ===========================================
# Main Start of OP2 Files Extraction == v1.1
# Remarks := Currently Working..
# Author := Upendra Reddy G
# ===========================================
barStressResults = pd.DataFrame({'A' : []})
barAxialStressResults = pd.DataFrame({'A' : []})
# quadFxStressResults = pd.DataFrame({'A' : []})
# quadFyStressResults = pd.DataFrame({'A' : []})
# quadFxyStressResults = pd.DataFrame({'A' : []})
rodAxialStressResults = pd.DataFrame({'A' : []})
shearStressResults = pd.DataFrame({'A' : []})
appendRodToBarResults = pd.DataFrame({'A' : []})
quad_tria_Fx_StressResults = pd.DataFrame({'A' : []})
quad_tria_Fy_StressResults = pd.DataFrame({'A' : []})
quad_tria_Fxy_StressResults = pd.DataFrame({'A' : []})

dictOp2List = ReadOP2Files('./')

# The Following loop will fetch results from every Available OP2 File
for key, op2File in dictOp2List.items():
    #print(key, op2File)
    op2Model = ReadOP2File(op2File)

    tempDF1 = ReadBarStressResults(op2Model)
    barStressResults = Join_DF(barStressResults, tempDF1)

    tempDF1 = ReadBarAxialStressResuls(op2Model)
    barAxialStressResults = Join_DF(barAxialStressResults, tempDF1)

#     tempDF1 = ReadShellFxResults(op2Model)
#     quadFxStressResults = Join_DF(quadFxStressResults, tempDF1)
    
#     tempDF1 = ReadShellFyResults(op2Model)
#     quadFyStressResults = Join_DF(quadFyStressResults, tempDF1)
    
#     tempDF1 = ReadShellFxyResults(op2Model)
#     quadFxyStressResults = Join_DF(quadFxyStressResults, tempDF1)

    tempDF1 = Read_Quad_and_Tria_Fx_Results(op2Model)
    quad_tria_Fx_StressResults = Join_DF(quad_tria_Fx_StressResults, tempDF1)
    
    tempDF1 = Read_Quad_and_Tria_Fy_Results(op2Model)
    quad_tria_Fy_StressResults = Join_DF(quad_tria_Fy_StressResults, tempDF1)
    
    tempDF1 = Read_Quad_and_Tria_Fxy_Results(op2Model)
    quad_tria_Fxy_StressResults = Join_DF(quad_tria_Fxy_StressResults, tempDF1)
    
    tempDF1 = ReadRodAxialStressResults(op2Model)
    rodAxialStressResults = Join_DF(rodAxialStressResults, tempDF1)
    
    tempDF1 = ReadShearStressResults(op2Model)
    shearStressResults = Join_DF(shearStressResults, tempDF1)    
    

#  The Following line of code will append Rod Axial Stress results to Bar Axial Stress Results    
appendRodToBarResults = barAxialStressResults.append(rodAxialStressResults, ignore_index=False)

# The Following lines of code will generate csv files for respective results.
barStressResults.to_csv('ALL_cBar_Stress.csv')
barAxialStressResults.to_csv('ALL_cBar_Stress_Axial.csv')
# quadFxStressResults.to_csv('ALL_cQUAD_Stress_Fx.csv')
# quadFyStressResults.to_csv('ALL_cQUAD_Stress_Fy.csv')
# quadFxyStressResults.to_csv('ALL_cQUAD_Stress_Fxy.csv')
quad_tria_Fx_StressResults.to_csv('ALL_cQUAD_cTRIA_Stress_Fx.csv')
quad_tria_Fy_StressResults.to_csv('ALL_cQUAD_cTRIA_Stress_Fy.csv')
quad_tria_Fxy_StressResults.to_csv('ALL_cQUAD_cTRIA_Stress_Fxy.csv')
rodAxialStressResults.to_csv('ALL_cRod_Stress_Axial.csv')
shearStressResults.to_csv('ALL_cShear_Stress.csv')
appendRodToBarResults.to_csv('ALL_appended_Rod_to_Bar_Stress_Axial.csv')

# End of Script.
print(' ======== ALL DONE SUCCESSFULLY ========== ')

Given File path is : 
 ./*.op2 

Found the following list of OP2 Files: 




C:\ProgramData\Anaconda3\lib\site-packages\pynastran-1.2.0+dev.no.checksum.error-py3.6.egg\pyNastran\op2\op2.py:665: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  obj.build_dataframe()


 ======== ALL DONE SUCCESSFULLY ========== 


# Test Section
## =============

In [ ]:
# op2Model = ReadOP2File('pc12-47-v1-6-static_post_m3.2_2102.op2')
# # cbush_Forces_DF = ReadcBushForces(op2Model)
# # cbush_Forces_DF.to_csv('ALL_CBUSH_Forces.csv')

In [ ]:
# # ===========================================
# # Main Start of OP2 Files Extraction == v1.0
# # Remarks := Currently Working..
# # Author := Upendra Reddy G
# # ===========================================
# barStressResults = pd.DataFrame({'A' : []})
# barAxialStressResults = pd.DataFrame({'A' : []})
# quadFxStressResults = pd.DataFrame({'A' : []})
# quadFyStressResults = pd.DataFrame({'A' : []})
# quadFxyStressResults = pd.DataFrame({'A' : []})
# rodAxialStressResults = pd.DataFrame({'A' : []})
# shearStressResults = pd.DataFrame({'A' : []})
# appendRodToBarResults = pd.DataFrame({'A' : []})

# dictOp2List = ReadOP2Files('./')

# # The Following loop will fetch results from every Available OP2 File
# for key, op2File in dictOp2List.items():
#     #print(key, op2File)
#     op2Model = ReadOP2File(op2File)

# #     tempDF1 = ReadBarStressResults(op2Model)
# #     barStressResults = Join_DF(barStressResults, tempDF1)

# #     tempDF1 = ReadBarAxialStressResuls(op2Model)
# #     barAxialStressResults = Join_DF(barAxialStressResults, tempDF1)

#     tempDF1 = Read_Quad_and_Tria_Fx_Results(op2Model)
#     quadFxStressResults = Join_DF(quadFxStressResults, tempDF1)
    
# #     tempDF1 = ReadShellFyResults(op2Model)
# #     quadFyStressResults = Join_DF(quadFyStressResults, tempDF1)
    
# #     tempDF1 = ReadShellFxyResults(op2Model)
# #     quadFxyStressResults = Join_DF(quadFxyStressResults, tempDF1)

In [ ]:
# quadDF = op2Model.cquad4_stress[92102].data_frame

In [ ]:
# # quadDF.head()
# quadDF = Add_Type(quadDF, 'CQUAD4')

In [ ]:
# quadDF.head()

In [ ]:
# triaDF = op2Model.ctria3_stress[92102].data_frame
# triaDF = Add_Type(triaDF, 'CTRIA3')
# triaDF.head()

In [ ]:
# tempDF = pd.concat([quadDF, triaDF], axis=0,sort=False)
# # tempDF1 = tempDF.reindex(columns=newQUADIndex_Fx(lc), level=1)

In [ ]:
# tempDF.tail()

In [ ]:
# tempDF = tempDF.reindex(columns=newQUADIndex_Fx(92102), level=1)
# tempDF.head()